# LOAY RASHID 2018102008

List of things done:
- Used pretrained resnet 50 with frozen layers, added 2 layers which were trained

- Applied a few basic transforms to the data

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torchvision
from torchvision import models
import torch.optim as optim
import pandas as pd
import numpy as np
import cv2
import torch.nn as nn
import torch.nn.functional as F

from sklearn import preprocessing
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
!wget -q https://datasets.aicrowd.com/default/aicrowd-practice-challenges/public/foodc/v0.1/train_images.zip
!wget -q https://datasets.aicrowd.com/default/aicrowd-practice-challenges/public/foodc/v0.1/test_images.zip
!wget -q https://datasets.aicrowd.com/default/aicrowd-practice-challenges/public/foodc/v0.1/train.csv
!wget -q https://datasets.aicrowd.com/default/aicrowd-practice-challenges/public/foodc/v0.1/test.csv


In [ ]:
!mkdir data
!mkdir data/test
!mkdir data/train
!unzip train_images -d data/train
!unzip test_images -d data/test

In [ ]:
class FoodData(Dataset):
    def __init__(self,data_list,data_dir = './',transform=None,train=True):
        super().__init__()
        self.data_list = data_list
        self.data_dir = data_dir
        self.transform = transform
        self.train = train
    
    def __len__(self):
        return self.data_list.shape[0]
    
    def __getitem__(self,item):
        if self.train:
          img_name,label = self.data_list.iloc[item]
        else:
          img_name = self.data_list.iloc[item]['ImageId']
        img_path = os.path.join(self.data_dir,img_name)
        img = cv2.imread(img_path,1)
        img = cv2.resize(img,(256,256))
        if self.transform is not None:
            img = self.transform(img)
        if self.train:
          return {
              'gt' : img,
              'label' : torch.tensor(label)

          }
        else:
          return {
              'gt':img
          }        

In [ ]:
train = pd.read_csv('train.csv')
le = preprocessing.LabelEncoder()
targets = le.fit_transform(train['ClassName'])
ntrain = train
ntrain['ClassName'] = targets  

In [ ]:
transforms_train = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(degrees=40, scale=(.9, 1.1), shear=0),
    transforms.ColorJitter(brightness=0.4,contrast=0.4,saturation=0.4),
    transforms.Normalize((0.3166, 0.3947, 0.4725), (0.1755, 0.1720, 0.1657))
])
train_path = 'data/train/train_images'
train_data = FoodData(data_list= ntrain,data_dir = train_path,transform = transforms_train)

In [ ]:
batch = 128
valid_size = 0.2
num = train_data.__len__()
# Dividing the indices for train and cross validation
indices = list(range(num))
np.random.shuffle(indices)
split = int(np.floor(valid_size*num))
train_idx,valid_idx = indices[split:], indices[:split]

#Create Samplers
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = DataLoader(train_data, batch_size = batch, sampler = train_sampler)
valid_loader = DataLoader(train_data, batch_size = batch, sampler = valid_sampler)

In [ ]:
transforms_test = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5) , (0.5,0.5,0.5))
])
test_path = 'data/test/test_images'
test = pd.read_csv('test.csv')
test_data = FoodData(data_list= test,data_dir = test_path,transform = transforms_test,train=False)

test_loader = DataLoader(test_data, batch_size=batch, shuffle=False)


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)
    

cuda:0


Here we define our model object along with our optimizer and error function. Typically for multi class classification we use `Cross Entropy Loss`. More about different types of losses are [here](https://pytorch.org/docs/stable/nn.html#loss-functions).   
We use the popular Adam optimizer with its default parameters. There are other optimizers like `SGD`, `RMSPROP`, `Adamax`,etc. You can have a detailed look at optimizers [here](https://pytorch.org/docs/stable/optim.html)

In [ ]:
model = Net().to(device)
error = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters())

In [ ]:
model = models.resnet50(pretrained=True).to(device)
    
for param in model.parameters():
    param.requires_grad = False   
    
model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 61)).to(device)

In [ ]:
error = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.fc.parameters())

In [ ]:
n_epochs = 5
valid_loss_min = np.Inf

train_losses = []
valid_losses = []

for epoch in range(n_epochs):
    train_loss = 0.0
    valid_loss = 0.0
    
    model.train()
    for images in train_loader:
        data = images['gt'].squeeze(0).to(device)
        # data = data.squeeze(0)
        target = images['label'].to(device)
#             clear the gradients of all optimized variables
        optimizer.zero_grad()
#         forward pass the model
        output = model(data)
#     backward pass the model
        loss = error(output,target)
        loss.backward()
#         Perform a single optimization step
        optimizer.step()
        train_loss += loss.item()*data.size(0)
        
    
    
    
    model.eval()
    for images in valid_loader:
        data = images['gt'].squeeze(0).to(device)
        target = images['label'].to(device)
#         forward pass now
        output = model(data)
#         calculate the branch loss
        loss = error(output, target)
#     update average validation loss
        valid_loss += loss.item()*data.size(0)
    
    train_loss /= len(train_loader.sampler)
    valid_loss /= len(valid_loader.sampler)
    
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    if valid_loss <= valid_loss_min:
        print("Validation Loss decreased {:0.6f} -> {:0.6f}".format(valid_loss_min,valid_loss))
        valid_loss_min = valid_loss
        torch.save(model.state_dict(), 'best_model_so_far.pth')

Epoch: 0 	Training Loss: 2.333293 	Validation Loss: 2.218895
Validation Loss decreased inf -> 2.218895
Epoch: 1 	Training Loss: 2.049859 	Validation Loss: 2.022505
Validation Loss decreased 2.218895 -> 2.022505
Epoch: 2 	Training Loss: 1.867413 	Validation Loss: 1.887063
Validation Loss decreased 2.022505 -> 1.887063
Epoch: 3 	Training Loss: 1.778317 	Validation Loss: 1.846909
Validation Loss decreased 1.887063 -> 1.846909
Epoch: 4 	Training Loss: 1.679658 	Validation Loss: 1.780733
Validation Loss decreased 1.846909 -> 1.780733


## Predict on Validation
Now we predict our trained model on the validation set and evaluate our model

In [ ]:
model.load_state_dict(torch.load('best_model_so_far.pth'))
model.eval() 
correct = 0
total = 0
pred_list = []
correct_list = []
with torch.no_grad():
    for images in valid_loader:
        data = images['gt'].squeeze(0).to(device)
        target = images['label'].to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        pr = predicted.detach().cpu().numpy()
        for i in pr:
          pred_list.append(i)
        tg = target.detach().cpu().numpy()
        for i in tg:
          correct_list.append(i)
        correct += (predicted == target).sum().item()

print('Accuracy of the network on the 10000 test images: %f %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 49.409871 %


## Evaluate the Performance
We use the same metrics as that will be used for the test set.  
[F1 score](https://en.wikipedia.org/wiki/F1_score) and [Log Loss](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.log_loss.html) are the metrics for this challenge

In [ ]:
from sklearn.metrics import f1_score,precision_score,log_loss   
print("F1 score :",f1_score(correct_list,pred_list,average='micro'))

F1 score : 0.49409871244635195


## Predict on test set
Time for the moment of truth! Predict on test set and time to make the submission.

In [ ]:
model.load_state_dict(torch.load('best_model_so_far.pth'))
model.eval()

preds = []
with torch.no_grad():
    for images in test_loader:
        data = images['gt'].squeeze(0).to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        pr = predicted.detach().cpu().numpy()
        for i in pr:
          preds.append(i)


## Save it in correct format

In [ ]:
# Create Submission file        
df = pd.DataFrame(le.inverse_transform(preds),columns=['ClassName'])
df.to_csv('submission.csv',index=False)

## To download the generated in collab csv run the below command

In [ ]:
from google.colab import files
files.download('submission.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Go to [platform](https://www.aicrowd.com/challenges/aicrowd-blitz-may-2020/problems/foodc). Participate in the challenge and submit the submission.csv.